In [1]:
# Update iterator algorithm; and making recommendations; 
import collections
import compress_pickle
import copy
import itertools
import libsbml
import numpy as np
import operator
import os
import pickle
import pandas as pd
import sys
import matplotlib.pyplot as plt
%matplotlib inline  

BIOMD_12 = 'BIOMD0000000012.xml'
BASE_DIR = '/Users/woosubshin/Desktop/AutomateAnnotation/'
DATA_DIR = os.path.join(BASE_DIR, "DATA")
ALGO_DIR = os.path.join(DATA_DIR, "algo")
CHEBI_DIR = os.path.join(DATA_DIR, "chebi")
RHEA_DIR = os.path.join(DATA_DIR, "rhea")
BIOMODEL_DIR = os.path.join(DATA_DIR, "biomodels/curated_biomodels_31mar2021")
BIGG_DIR = '/Users/woosubshin/Desktop/AutomateAnnotation/DATA/bigg'
ecoli_fpath = os.path.join(BIGG_DIR, "e_coli_core.xml")

PROJ_DIR = os.path.join(os.getcwd(), os.pardir)
AMAS_DIR = os.path.join(PROJ_DIR, "AMAS")
sys.path.append(PROJ_DIR)

SUPPL_DIR = os.path.join(PROJ_DIR, os.pardir, "AMAS_suppl")
ACCURACY_DIR = os.path.join(SUPPL_DIR, "data_for_credibility")

from AMAS import annotation_maker as am
from AMAS import species_annotation as sa
from AMAS import reaction_annotation as ra
from AMAS import recommender
from AMAS import constants as cn
from AMAS import iterator as it
from AMAS import tools

spec_ml_df = pd.read_csv(os.path.join(ACCURACY_DIR, 'biomd_individual_species_accuracy.csv'))
biomds_spec = np.unique(spec_ml_df['model'])
print("Number of BioModels to be analyzed for SPECIES: %d" % len(biomds_spec))

reac_ml_df = pd.read_csv(os.path.join(ACCURACY_DIR, 'biomd_individual_reactions_accuracy.csv'))
biomds_reac = np.unique(reac_ml_df['model'])
print("Number of BioModels to be analyzed for REACTIONS: %d" % len(biomds_reac))

Number of BioModels to be analyzed for SPECIES: 306
Number of BioModels to be analyzed for REACTIONS: 131


In [2]:
one_fpath = os.path.join(BIOMODEL_DIR, 'BIOMD0000000190.xml')
recom = recommender.Recommender(libsbml_fpath=one_fpath)

In [3]:
recom.recommendSpecies(min_len=20)

,file,type,id,display name,meta id,annotation,annotation label,match score,existing,UPDATE ANNOTATION
0,BIOMD0000000190.xml,species,SAM,S-adenosyl-L-methionine,metaid_0000036,CHEBI:15414,S-adenosyl-L-methionine,1.0,1,keep
1,BIOMD0000000190.xml,species,SAM,S-adenosyl-L-methionine,metaid_0000036,CHEBI:59789,S-adenosyl-L-methionine zwitterion,1.0,0,ignore
2,BIOMD0000000190.xml,species,A,S-adenosylmethioninamine,metaid_0000037,CHEBI:15625,S-adenosylmethioninamine,1.0,1,keep


In [4]:
tools.formatRhea('abc')

AttributeError: module 'AMAS.tools' has no attribute 'formatRhea'

In [4]:
# def getSBMLDocument(sbml_document,
#                     chosen,
#                     auto_feedback=False):
#   """
#   Create an SBML document 
#   based on the feedback.
#   If auto_feedback is checked,
  
  
#   Parameters
#   ----------
#   sbml_document: libsbml.SBMLDocument
  
#   chosen: pandas.DataFrame
  
#   Returns
#   -------
#   str
#       SBML document
#   """
#   model = sbml_document.getModel()
#   if auto_feedback:
#     chosen.replace('ignore', 'add', inplace=True)
#   ELEMENT_FUNC = {'species': model.getSpecies,
#                   'reaction': model.getReaction}
#   element_types = list(np.unique(chosen['type']))
#   for one_type in element_types:
#     maker = am.AnnotationMaker(one_type)
#     ACTION_FUNC = {'delete': maker.deleteAnnotation,
#                    'add': maker.addAnnotation}
#     df_type = chosen[chosen['type']==one_type]
#     uids = list(np.unique(df_type['id']))
#     meta_ids = {val:list(df_type[df_type['id']==val]['meta id'])[0] for val in uids}
#     # going through one id at a time
#     for one_id in uids:
#       orig_str = ELEMENT_FUNC[one_type](one_id).getAnnotationString()
#       df_id = df_type[df_type['id']==one_id]
#       dels = list(df_id[df_id[cn.DF_UPDATE_ANNOTATION_COL]=='delete'].loc[:, 'annotation'])
#       adds_raw = list(df_id[df_id[cn.DF_UPDATE_ANNOTATION_COL]=='add'].loc[:, 'annotation'])
#       # existing annotations to be kept 
#       keeps = list(df_id[df_id[cn.DF_UPDATE_ANNOTATION_COL]=='keep'].loc[:, 'annotation'])
#       adds = list(set(adds_raw + keeps))
#       # if type is 'reaction', need to map rhea terms back to ec/kegg terms to delete them. 
#       if one_type == 'reaction':
#         rhea_del_terms = list(set(itertools.chain(*[tools.getAssociatedTermsToRhea(val) for val in dels])))
#         deled = maker.deleteAnnotation(rhea_del_terms, orig_str)
#       elif one_type == 'species':
#         deled = maker.deleteAnnotation(dels, orig_str)
#       added = maker.addAnnotation(adds, deled, meta_ids[one_id])
#       ELEMENT_FUNC[one_type](one_id).setAnnotation(added)
#   return sbml_document

In [5]:
recommender.Recommender(libsbml_fpath=one_fpath)

In [33]:
# developing recommendAnnotation method
mssc = 'top'
cutoff = 0
optimize = 'y'
outtype = 'table'

pred_spec = recom.recommendSpecies(ids=None, 
                                   min_len=0,
                                   mssc='top',
                                   cutoff=0.0,
                                   outtype='table')

pred_reac = recom.recommendReactions(ids=None, 
                                     min_len=0,
                                     mssc='top',
                                     cutoff=0.0,
                                     outtype='table')

pred_comb = pd.concat([pred_spec, pred_reac], ignore_index=True)

pred_sbml = recom.getSBMLDocument(sbml_document=recom.sbml_document,
                                  chosen=pred_comb,
                                  auto_feedback=True)
pred_sbml

<SBMLDocument>

In [34]:
model = pred_sbml.getModel()
upd_spec_anot = tools.extractExistingSpeciesAnnotation(model)
upd_reac_anot = tools.extractExistingReactionAnnotation(model)
print(upd_spec_anot)
print(upd_reac_anot)

{'SAM': ['CHEBI:15414', 'CHEBI:59789'], 'A': ['CHEBI:15625'], 'P': ['CHEBI:17148', 'CHEBI:326268'], 'S': ['CHEBI:15746', 'CHEBI:45725'], 'D': ['CHEBI:16610', 'CHEBI:57834'], 'aS': ['CHEBI:17312', 'CHEBI:58101'], 'aD': ['CHEBI:22204', 'CHEBI:17927', 'CHEBI:58324'], 'Met': ['CHEBI:16643', 'CHEBI:64558', 'CHEBI:16811'], 'ORN': ['CHEBI:15729', 'CHEBI:46911'], 'AcCoA': ['CHEBI:15351', 'CHEBI:57288'], 'CoA': ['CHEBI:15346', 'CHEBI:48076', 'CHEBI:57287']}
{'ODC': ['RHEA:22964'], 'SAMdc': ['RHEA:15981'], 'SSAT_for_S': ['RHEA:33099', 'RHEA:11116', 'RHEA:28150', 'RHEA:28270'], 'SSAT_for_D': ['RHEA:33099', 'RHEA:11116', 'RHEA:28150', 'RHEA:28270'], 'PAO_for_aD': ['RHEA:16133', 'RHEA:25800'], 'PAO_for_aS': ['RHEA:16133', 'RHEA:25800'], 'SpdS': ['RHEA:12721'], 'SpmS': ['RHEA:19973'], 'MAT': ['RHEA:21080']}


In [41]:
tools.extractRheaFromAnnotationString(mat_anot)

['RHEA:21080']

In [44]:
'RHEA:61020' in cn.REF_RHEA2MASTER

True

In [46]:
inp_str = mat_anot

mat_anot = model.getReaction('MAT').getAnnotationString()
tools.getQualifierFromString(inp_str, cn.RHEA)

exist_rheas = [cn.RHEA_HEADER+val for val in tools.getQualifierFromString(inp_str, cn.RHEA)]
map_rhea_bis = [cn.REF_RHEA2MASTER[val] for val in exist_rheas if val in cn.REF_RHEA2MASTER.keys()]
print(map_rhea_bis)

[]


In [51]:
def formatRhea(one_rhea):
  """
  Format rhea values; 
  if 'RHEA:' is not in the name,
  add it; if not, ignore it
  
  Parameters
  ----------
  str: one_rhea
  
  Returns
  -------
  :str
  """
  if one_rhea[:4].lower() == 'rhea':
    str_to_add = one_rhea[5:] 
  else:
    str_to_add = one_rhea
  return cn.RHEA_HEADER + str_to_add

In [53]:
formatRhea('12344')

'RHEA:12344'

In [50]:
cn.RHEA_HEADER.lower()

'rhea:'

In [48]:
for val in exist_rheas:
  print(cn.REF_RHEA2MASTER[val])

KeyError: 'RHEA:RHEA:15425'

In [32]:
sbml_document=recom.sbml_document
chosen = pred_comb
auto_feedback=True

model = sbml_document.getModel()
if auto_feedback:
  chosen.replace('ignore', 'add', inplace=True)
ELEMENT_FUNC = {'species': model.getSpecies,
                'reaction': model.getReaction}
element_types = list(np.unique(chosen['type']))
for one_type in element_types:
  print(one_type)
  maker = am.AnnotationMaker(one_type)
  ACTION_FUNC = {'delete': maker.deleteAnnotation,
                 'add': maker.addAnnotation}
  df_type = chosen[chosen['type']==one_type]
  uids = list(np.unique(df_type['id']))
  meta_ids = {val:list(df_type[df_type['id']==val]['meta id'])[0] for val in uids}
  print(meta_ids)
  # going through one id at a time
  for one_id in uids:
    orig_str = ELEMENT_FUNC[one_type](one_id).getAnnotationString()
    df_id = df_type[df_type['id']==one_id]
    dels = list(df_id[df_id[cn.DF_UPDATE_ANNOTATION_COL]=='delete'].loc[:, 'annotation'])
    adds_raw = list(df_id[df_id[cn.DF_UPDATE_ANNOTATION_COL]=='add'].loc[:, 'annotation'])
    # existing annotations to be kept 
    keeps = list(df_id[df_id[cn.DF_UPDATE_ANNOTATION_COL]=='keep'].loc[:, 'annotation'])
    adds = list(set(adds_raw + keeps))
    # if type is 'reaction', need to map rhea terms back to ec/kegg terms to delete them. 
    if one_type == 'reaction':
      rhea_del_terms = list(set(itertools.chain(*[tools.getAssociatedTermsToRhea(val) for val in dels])))
      deled = maker.deleteAnnotation(rhea_del_terms, orig_str)
    elif one_type == 'species':
      deled = maker.deleteAnnotation(dels, orig_str)
    added = maker.addAnnotation(adds, deled, meta_ids[one_id])
    print(added)
    ELEMENT_FUNC[one_type](one_id).setAnnotation(added)

reaction
{'MAT': 'metaid_0000070', 'ODC': 'metaid_0000062', 'PAO_for_aD': 'metaid_0000066', 'PAO_for_aS': 'metaid_0000067', 'P_efflux': 'metaid_0000073', 'SAMdc': 'metaid_0000063', 'SSAT_for_D': 'metaid_0000065', 'SSAT_for_S': 'metaid_0000064', 'SpdS': 'metaid_0000068', 'SpmS': 'metaid_0000069', 'VCoA': 'metaid_0000071', 'VacCoA': 'metaid_0000072', 'aD_efflux': 'metaid_0000074'}
<annotation>
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:vCard="http://www.w3.org/2001/vcard-rdf/3.0#" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/" xmlns:bqmodel="http://biomodels.net/model-qualifiers/">
    <rdf:Description rdf:about="#metaid_0000070">
      <bqbiol:isVersionOf>
        <rdf:Bag>
          <rdf:li rdf:resource="http://identifiers.org/ec-code/2.5.1.6"/>
          <rdf:li rdf:resource="http://identifiers.org/kegg.reaction/R00177"/>
          <rdf:li rdf:resource="http://i

In [28]:
meta_ids

{'A': 'metaid_0000037',
 'AcCoA': 'metaid_0000045',
 'CoA': 'metaid_0000046',
 'D': 'metaid_0000040',
 'Met': 'metaid_0000043',
 'ORN': 'metaid_0000044',
 'P': 'metaid_0000038',
 'S': 'metaid_0000039',
 'SAM': 'metaid_0000036',
 'aD': 'metaid_0000042',
 'aS': 'metaid_0000041'}

In [22]:
pred_comb['UPDATE ANNOTATION']

0      keep
1       add
2      keep
3      keep
4       add
       ... 
147     add
148     add
149     add
150     add
151     add
Name: UPDATE ANNOTATION, Length: 152, dtype: object

In [14]:
type(pred_reac)

pandas.core.frame.DataFrame

In [ ]:
def recommendAnnotation(mssc='top',
                        cutoff=0.0,
                        optimize=False,
                        outtype='table')
  """
  Recommend annotations of both 
  species and reactions.
  Uses self.recommendSpecies and
  self.recommendReactions.
  
  Parameters
  ----------
  mssc: str
  cutoff: float
  optimize: bool
      If True, optimize
      by iteration 
  outtype: str
      either 'table' or 'sbml'
      If 'sbml' is chosen,
      returns updated model file
      with annotations. 
  
  Returns
  -------
  : pandas.DataFrame or string
  """
  pass

In [8]:
# testing getSBMLDocument method

mssc = 'top'
cutoff = 0.0

pred = recom.recommendSpecies(ids=None,
                              mssc=mssc,
                              cutoff=cutoff)

res_doc = recom.getSBMLDocument(sbml_document=recom.sbml_document,
                                chosen=pred,
                                auto_feedback=True)
model = res_doc.getModel()


upd_spec_anot = tools.extractExistingSpeciesAnnotation(model)
print(upd_spec_anot)
print(upd_spec_anot['SAM'] == ['CHEBI:15414', 'CHEBI:59789'])

upd_reac_anot = tools.extractExistingReactionAnnotation(model)
print(upd_reac_anot)


# for s in model.getListOfSpecies():
#   print(s.getAnnotationString())

# for r in model.getListOfReactions():
#   print(r.getAnnotationString())

# Will need to convert this to string
# res = libsbml.writeSBMLToString(res_doc)
# res

{'SAM': ['CHEBI:15414', 'CHEBI:59789'], 'A': ['CHEBI:15625'], 'P': ['CHEBI:17148', 'CHEBI:326268'], 'S': ['CHEBI:15746', 'CHEBI:45725'], 'D': ['CHEBI:16610', 'CHEBI:57834'], 'aS': ['CHEBI:17312', 'CHEBI:58101'], 'aD': ['CHEBI:22204', 'CHEBI:58324', 'CHEBI:17927'], 'Met': ['CHEBI:16643', 'CHEBI:64558', 'CHEBI:16811'], 'ORN': ['CHEBI:15729', 'CHEBI:46911'], 'AcCoA': ['CHEBI:15351', 'CHEBI:57288'], 'CoA': ['CHEBI:15346', 'CHEBI:48076', 'CHEBI:57287']}
{'ODC': ['RHEA:22964'], 'SAMdc': ['RHEA:15981'], 'SSAT_for_S': ['RHEA:28150', 'RHEA:33099', 'RHEA:11116', 'RHEA:28270'], 'SSAT_for_D': ['RHEA:28150', 'RHEA:33099', 'RHEA:11116', 'RHEA:28270'], 'PAO_for_aD': ['RHEA:25800', 'RHEA:16133'], 'PAO_for_aS': ['RHEA:25800', 'RHEA:16133'], 'SpdS': ['RHEA:12721'], 'SpmS': ['RHEA:19973'], 'MAT': ['RHEA:21080']}


In [9]:
pred

,file,type,id,display name,meta id,annotation,annotation label,match score,existing,UPDATE ANNOTATION
0,BIOMD0000000190.xml,species,SAM,S-adenosyl-L-methionine,metaid_0000036,CHEBI:15414,S-adenosyl-L-methionine,1.000,1,keep
1,BIOMD0000000190.xml,species,SAM,S-adenosyl-L-methionine,metaid_0000036,CHEBI:59789,S-adenosyl-L-methionine zwitterion,1.000,0,add
2,BIOMD0000000190.xml,species,A,S-adenosylmethioninamine,metaid_0000037,CHEBI:15625,S-adenosylmethioninamine,1.000,1,keep
3,BIOMD0000000190.xml,species,P,Putrescine,metaid_0000038,CHEBI:17148,putrescine,1.000,1,keep
4,BIOMD0000000190.xml,species,P,Putrescine,metaid_0000038,CHEBI:326268,"1,4-butanediammonium",1.000,0,add
5,BIOMD0000000190.xml,species,S,Spermine,metaid_0000039,CHEBI:15746,spermine,1.000,1,keep
6,BIOMD0000000190.xml,species,S,Spermine,metaid_0000039,CHEBI:45725,spermine(4+),1.000,0,add
7,BIOMD0000000190.xml,species,D,Spermidine,metaid_0000040,CHEBI:16610,spermidine,1.000,1,keep
8,BIOMD0000000190.xml,species,D,Spermidine,metaid_0000040,CHEBI:57834,spermidine(3+),1.000,0,add
9,BIOMD0000000190.xml,species,aS,N1-Acetylspermine,metaid_0000041,CHEBI:17312,N(1)-acetylspermine,1.000,1,keep


In [7]:
mssc = 'top'
cutoff = 0.0

pred = recom.recommendSpecies(ids=None,
                              mssc=mssc,
                              cutoff=cutoff)


In [2]:
# res_doc = getSBMLDoc(sbml_document=recom.sbml_document,
#                  chosen=pred,
#                  auto_feedback=True)
# model = res_doc.getModel()
# # for s in model.getListOfSpecies():
# #   print(s.getAnnotationString())

# # for r in model.getListOfReactions():
# #   print(r.getAnnotationString())

# res = libsbml.writeSBMLToString(res_doc)
# # print(res)

In [ ]:
## replacing (i.e., auto-feedback)

# getSBMLDoc
ELEMENT_FUNC = {'species': model.getSpecies,
                'reaction': model.getReaction}
# arguments  for the method
elemnt_type = 'species'
recommended = pred

# main body
model = recom.sbml_document.getModel()
maker = am.AnnotationMaker(element_type)
ACTION_FUNC = {'delete': maker.deleteAnnotation,
               'add': maker.addAnnotation}


In [15]:
# def getSBMLDoc(sbml_document,
#                chosen,
#                auto_feedback=False):
#   """
#   Create an SBML document 
#   based on the feedback.
#   If auto_feedback is checked,
  
  
#   Parameters
#   ----------
#   sbml_document: libsbml.SBMLDocument
  
#   chosen: pandas.DataFrame
  
#   Returns
#   -------
#   str
#       SBML document
#   """
#   model = sbml_document.getModel()
#   if auto_feedback:
#     chosen.replace('ignore', 'add', inplace=True)
#   ELEMENT_FUNC = {'species': model.getSpecies,
#                   'reaction': model.getReaction}
#   element_types = list(np.unique(chosen['type']))
#   for one_type in element_types:
#     maker = am.AnnotationMaker(one_type)
#     ACTION_FUNC = {'delete': maker.deleteAnnotation,
#                    'add': maker.addAnnotation}
#     df_type = chosen[chosen['type']==one_type]
#     uids = list(np.unique(df_type['id']))
#     meta_ids = {val:list(df_type[df_type['id']==val]['meta id'])[0] for val in uids}
#     # going through one id at a time
#     for one_id in uids:
#       orig_str = ELEMENT_FUNC[one_type](one_id).getAnnotationString()
#       df_id = df_type[df_type['id']==one_id]
#       dels = list(df_id[df_id[cn.DF_UPDATE_ANNOTATION_COL]=='delete'].loc[:, 'annotation'])
#       adds_raw = list(df_id[df_id[cn.DF_UPDATE_ANNOTATION_COL]=='add'].loc[:, 'annotation'])
#       # existing annotations to be kept 
#       keeps = list(df_id[df_id[cn.DF_UPDATE_ANNOTATION_COL]=='keep'].loc[:, 'annotation'])
#       adds = list(set(adds_raw + keeps))
#       # if type is 'reaction', need to map rhea terms back to ec/kegg terms to delete them. 
#       if one_type == 'reaction':
#         rhea_del_terms = list(set(itertools.chain(*[tools.getAssociatedTermsToRhea(val) for val in dels])))
#         deled = maker.deleteAnnotation(rhea_del_terms, orig_str)
#       elif one_type == 'species':
#         deled = maker.deleteAnnotation(dels, orig_str)
#       added = maker.addAnnotation(adds, deled, meta_ids[one_id])
#       ELEMENT_FUNC[one_type](one_id).setAnnotation(added)
#   return sbml_document

In [ ]:



# # dictionary with empty lists; 
# saved_elements = {k:[] for k in ELEMENT_TYPES}
# for one_type in ELEMENT_TYPES:
    
#   type_selection = recom.selection[one_type]
#   maker = am.AnnotationMaker(one_type)
    
    
#   sel2save = type_selection
#   for one_k in sel2save.keys():
#     one_element = ELEMENT_FUNC[one_type](one_k)
#     meta_id = one_element.meta_id
#     df = sel2save[one_k]
#     cands2save = list(df['annotation'])
#     if cands2save:
#       if option == 'augment':
#         orig_annotation = one_element.getAnnotationString()
#         annotation_str = maker.addAnnotation(cands2save,
#                                              orig_annotation,
#                                              meta_id)
#       elif option == 'replace':
#         annotation_str = maker.getAnnotationString(cands2save, meta_id)
#       one_element.setAnnotation(annotation_str)
#       saved_elements[one_type].append(one_k)
#     else:
#       continue

TypeError: Recommender.recommendReactions() got an unexpected keyword argument 'outtype'

In [6]:
rec = recom.recommendReactions(ids=['ODC', 'SAMdc'])
rec

,file,type,id,display name,meta id,annotation,annotation label,match score,existing,UPDATE ANNOTATION
0,BIOMD0000000190.xml,reaction,ODC,Ornithine decarboxylase,metaid_0000062,RHEA:22964,ornithine decarboxylase activity,1.0,1,keep
1,BIOMD0000000190.xml,reaction,ODC,Ornithine decarboxylase,metaid_0000062,RHEA:28827,L-ornithine(out) + putrescine(in) = L-ornithine(in) + putrescine(out),1.0,0,ignore
2,BIOMD0000000190.xml,reaction,ODC,Ornithine decarboxylase,metaid_0000062,RHEA:59048,D-ornithine + H(+) = CO2 + putrescine,1.0,0,ignore
3,BIOMD0000000190.xml,reaction,SAMdc,S-adenosylmethionine decarboxylase,metaid_0000063,RHEA:15981,adenosylmethionine decarboxylase activity,0.5,1,keep


In [7]:
spec_res = recom.getSpeciesListRecommendation(pred_ids=['SAM', 'ORN'],
                                              get_df=True)

In [8]:
spec_res

[      annotation  match score                               label
 SAM                                                              
 1    CHEBI:15414          1.0             S-adenosyl-L-methionine
 2    CHEBI:59789          1.0  S-adenosyl-L-methionine zwitterion,
       annotation  match score              label
 ORN                                             
 1    CHEBI:15729          1.0        L-ornithine
 2    CHEBI:46911          1.0  L-ornithinium(1+)]

In [9]:
spec_rec = recom.recommendSpecies(['SAM', 'ORN'])
spec_rec

,file,type,id,display name,meta id,annotation,annotation label,match score,existing,UPDATE ANNOTATION
0,BIOMD0000000190.xml,species,SAM,S-adenosyl-L-methionine,metaid_0000036,CHEBI:15414,S-adenosyl-L-methionine,1.0,1,keep
1,BIOMD0000000190.xml,species,SAM,S-adenosyl-L-methionine,metaid_0000036,CHEBI:59789,S-adenosyl-L-methionine zwitterion,1.0,0,ignore
2,BIOMD0000000190.xml,species,ORN,L-Ornithine,metaid_0000044,CHEBI:15729,L-ornithine,1.0,1,keep
3,BIOMD0000000190.xml,species,ORN,L-Ornithine,metaid_0000044,CHEBI:46911,L-ornithinium(1+),1.0,0,ignore


In [10]:
specs = recom.getSpeciesIDs()
print("...\nAnalyzing %d species...\n" % len(specs))
res_spec = recom.getSpeciesListRecommendation(pred_ids=specs)
cur_form = dict()
for one_rec in res_spec:
  cands = [val[0] for val in one_rec.candidates]
  one_form = list(set([cn.REF_CHEBI2FORMULA[k] \
                    for k in cands if k in cn.REF_CHEBI2FORMULA.keys()]))
  cur_form[one_rec.id] = one_form

reacts = recom.getReactionIDs()
print("...\nAnalyzing %d reaction(s)...\n" % len(reacts))
res_reac = recom.getReactionListRecommendation(pred_ids=reacts,
                                             spec_res=res_spec)

...
Analyzing 11 species...

...
Analyzing 13 reaction(s)...



In [11]:
cur_form

{'SAM': ['C15N6O5S'],
 'A': ['C14N6O3S'],
 'P': ['C4N2'],
 'S': ['C10N4'],
 'D': ['C7N3'],
 'aS': ['C12N4O'],
 'aD': ['C9N3O'],
 'Met': ['C5NO2S'],
 'ORN': ['C5N2O2'],
 'AcCoA': ['C23N7O17P3S'],
 'CoA': ['C21N7O16P3S', 'C2O2']}

In [12]:
type(reacts)

list

In [13]:
# def optimizePrediction(cur_spec_formula,
#                        reaction_cl,
#                        reactions_to_update):
#   """
#   Optimize prediction using iteration.
  
#   Parameters
#   ----------
#   cur_spec_formula: dict
#       (predicted) formula of species

#   reaction_cl: AMAS.reaction_annotation.ReactionAnnotation
  
#   reactions_to_update: list
#       IDs of reactions
      
#   Returns
#   -------
#   fin_spec_recom: Recommendation (namedtuple)
  
#   fin_reac_recom: Recommendation (namedtuple)
#   """
#   anot_iter = it.Iterator(cur_spec_formula=cur_spec_formula,
#                           reaction_cl=reaction_cl,
#                           reactions_to_update=reactions_to_update)
#   res_iter = anot_iter.match()
#   recoms_tobe_added = []
#   for one_spec in res_iter.keys():
#     pred_reacs = [val.id for val in res_reac]
#     reacs_using_one_spec = [val for val in pred_reacs \
#                             if one_spec in reaction_cl.reaction_components[val]]
#     filt_res_reac = [val for val in res_reac if val.id in reacs_using_one_spec]
#     # match score of reactions using that species
#     adj_match_score = np.mean(list(itertools.chain(*[[cand[1] for cand in val.candidates] \
#                               for val in filt_res_reac])))
#     cands = res_iter[one_spec]
#     adj_formulas = list(set([cn.REF_CHEBI2FORMULA[k] \
#                              for k in cands if k in cn.REF_CHEBI2FORMULA.keys()]))
#     urls = [cn.CHEBI_DEFAULT_URL + val[6:] for val in cands]
#     labels = [cn.REF_CHEBI2LABEL[val] for val in cands]
#     adj_recom = cn.Recommendation(one_spec,
#                                   [(val, adj_match_score) for val in cands],
#                                   urls,
#                                   labels)
#     recoms_tobe_added.append(adj_recom)
#   fin_spec_recom = recoms_tobe_added + \
#                    [val for val in res_spec if val.id not in res_iter.keys()]
#   fin_reac_recom = recom.getReactionListRecommendation(pred_ids=reacts,
#                                                        spec_res=fin_spec_recom)
#   return fin_spec_recom, fin_reac_recom

In [5]:
for idx, one_biomd in enumerate(['BIOMD0000000017.xml']):
  if idx % 20 == 0:
    print("we are at", idx)
  
  one_fpath = os.path.join(BIOMODEL_DIR, one_biomd)
  recom = recommender.Recommender(libsbml_fpath=one_fpath)

  specs = recom.getSpeciesIDs()
  res_spec = recom.getSpeciesListRecommendation(pred_ids=specs)
  # cur_form = dict()
  # for one_rec in res_spec:
  #   cur_cands = [val[0] for val in one_rec.candidates]
  #   one_form = list(set([cn.REF_CHEBI2FORMULA[k] \
  #                     for k in cur_cands if k in cn.REF_CHEBI2FORMULA.keys()]))
  #   cur_form[one_rec.id] = one_form

  reacts = recom.getReactionIDs() 
  res_reac = recom.getReactionListRecommendation(pred_ids=reacts,
                                                 spec_res=res_spec)
  opt_spec_recom, opt_reac_recom = recom.optimizePrediction(pred_spec=res_spec,
                                                            pred_reac=res_reac,
                                                            reactions_to_update=reacts)
  oner = opt_spec_recom!=res_spec
  twor = opt_reac_recom!=res_reac
  if any([oner, twor]):
    print("Found!")
    break

we are at 0
Found!


In [9]:
[val for val in res_spec if val.id == 'AcetoinIn']

[Recommendation(id='AcetoinIn', candidates=[('CHEBI:2430', 1.0)], urls=['https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI%3A2430'], labels=['aconitine'])]

In [10]:
[val for val in opt_spec_recom if val.id == 'AcetoinIn']

[Recommendation(id='AcetoinIn', candidates=[('CHEBI:15378', 0.98792749244713)], urls=['https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI%3A15378'], labels=['hydron'])]

In [3]:
for idx, one_biomd in enumerate(biomds_spec[3:5]):
  if idx % 20 == 0:
    print("we are at", idx)
  
  one_fpath = os.path.join(BIOMODEL_DIR, one_biomd)
  recom = recommender.Recommender(libsbml_fpath=one_fpath)

  specs = recom.getSpeciesIDs()
  res_spec = recom.getSpeciesListRecommendation(pred_ids=specs)
  cur_form = dict()
  for one_rec in res_spec:
    cur_cands = [val[0] for val in one_rec.candidates]
    one_form = list(set([cn.REF_CHEBI2FORMULA[k] \
                      for k in cur_cands if k in cn.REF_CHEBI2FORMULA.keys()]))
    cur_form[one_rec.id] = one_form

  reacts = recom.getReactionIDs() 
  res_reac = recom.getReactionListRecommendation(pred_ids=reacts,
                                                 spec_res=res_spec)
  ## 
  anot_iter = it.Iterator(cur_spec_formula=cur_form,
                          reaction_cl=recom.reactions,
                          reactions_to_update=reacts)
  res_iter = anot_iter.match()
  recoms_tobe_added = []
  for one_spec in res_iter.keys():
    pred_reacs = [val.id for val in res_reac]
    reacs_using_one_spec = [val for val in pred_reacs \
                            if one_spec in recom.reactions.reaction_components[val]]
    filt_res_reac = [val for val in res_reac if val.id in reacs_using_one_spec]
    # match score of reactions using that species
    adj_match_score = np.mean(list(itertools.chain(*[[cand[1] for cand in val.candidates] \
                              for val in filt_res_reac])))
    cands = res_iter[one_spec]
    adj_formulas = list(set([cn.REF_CHEBI2FORMULA[k] \
                             for k in cands if k in cn.REF_CHEBI2FORMULA.keys()]))
    urls = [cn.CHEBI_DEFAULT_URL + val[6:] for val in cands]
    labels = [cn.REF_CHEBI2LABEL[val] for val in cands]
    adj_recom = cn.Recommendation(one_spec,
                                  [(val, adj_match_score) for val in cands],
                                  urls,
                                  labels)
    recoms_tobe_added.append(adj_recom)
  fin_spec_recom = recoms_tobe_added + \
                   [val for val in res_spec if val.id not in res_iter.keys()]
  fin_reac_recom = recom.getReactionListRecommendation(pred_ids=reacts,
                                                       spec_res=fin_spec_recom)
  oner = fin_spec_recom!=res_spec
  twor = fin_reac_recom!=res_reac
  if any([oner, twor]):
    print("Found!")
    break

we are at 0
Found!


In [57]:
idx

3

In [22]:
res_iter

{}

In [ ]:
# combined

In [ ]:
def recommendAnnotation(mssc='top',
                        cutoff=0.0,
                        outtype='table'):
  """
  Recommend annotation of 
  """

In [13]:
recom.species.getNameToUse('SAM')

'S-adenosyl-L-methionine'

In [19]:
# getRecomSBML
model = recom.sbml_document.getModel()
ELEMENT_FUNC = {'species': model.getSpecies,
                'reaction': model.getReaction}
#######################
one_type = 'species'
one_recomt = spec_recomt
#######################
maker = am.AnnotationMaker(one_type)

In [ ]:
model = self.sbml_document.getModel()
ELEMENT_FUNC = {'species': model.getSpecies,
                'reaction': model.getReaction}
# dictionary with empty lists; 
saved_elements = {k:[] for k in ELEMENT_TYPES}
for one_type in ELEMENT_TYPES:
  type_selection = self.selection[one_type]
  maker = am.AnnotationMaker(one_type)
  sel2save = type_selection


  for one_k in sel2save.keys():
    one_element = ELEMENT_FUNC[one_type](one_k)
    meta_id = one_element.meta_id
    df = sel2save[one_k]
    cands2save = list(df['annotation'])
    if cands2save:
      if option == 'augment':
        orig_annotation = one_element.getAnnotationString()
        annotation_str = maker.addAnnotation(cands2save,
                                             orig_annotation,
                                             meta_id)
      elif option == 'replace':
        annotation_str = maker.getAnnotationString(cands2save, meta_id)
      one_element.setAnnotation(annotation_str)
      saved_elements[one_type].append(one_k)
    else:
      continue